# Comformer D2R2 Single Property Prediction Notebook

This notebook generates predictions for the single property model, performs de-standardization, and calculates metrics.

In [1]:
import pandas as pd
import numpy as np
import json
import os
import torch
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

# Add project root to path
current_dir = os.path.dirname(os.path.abspath('.'))
if current_dir not in sys.path:
    sys.path.append(current_dir)

In [2]:
import torch
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should be >0
print(torch.cuda.current_device())  # Should print device ID
print(torch.__version__)  # Check PyTorch version
print(torch.version.cuda) # Should be 11.7, but CUDA system is 12.1
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.get_device_name(0))  # Should print GPU name


True
1
0
2.5.1
12.1
True
NVIDIA L40S


## 1. Load Data and Model

First, we need to load the ground truth data and test-train split.

In [3]:
# Define paths
data_path = os.path.join(current_dir, 'data', 'DFT_data.csv')
split_path = os.path.join(current_dir, 'output', 'D2R2_WF_bottom', 'ids_train_val_test.json')
model_path = os.path.join(current_dir, 'output', 'D2R2_WF_bottom', 'checkpoint_395.pt')

# Load data
data_df = pd.read_csv(data_path)

# Load the train/val/test split information
with open(split_path, 'r') as f:
    train_test_val = json.loads(f.read())

# Create ID column in the data dataframe to match with prediction IDs
data_df['id'] = data_df["mpid"].astype(str) + data_df["miller"].astype(str) + data_df["term"].astype(str)

# Filter data by sets
train_data_df = data_df[data_df['id'].isin(train_test_val['id_train'])]
val_data_df = data_df[data_df['id'].isin(train_test_val['id_val'])]
test_data_df = data_df[data_df['id'].isin(train_test_val['id_test'])]

# Get counts
print(f"Number of training samples: {len(train_data_df)}")
print(f"Number of validation samples: {len(val_data_df)}")
print(f"Number of test samples: {len(test_data_df)}")

Number of training samples: 29481
Number of validation samples: 3685
Number of test samples: 3685


## 2. Load and Initialize the Model

Now we'll load the trained model to generate predictions.

In [7]:
# Import necessary modules
from ..comformer.models.comformer import iComformer, iComformerConfig
from ..comformer.data import get_torch_test_loader

# Load the model configuration
config = iComformerConfig(
    name="iComformer",
    output_features=1,  # Single property model
    use_angle=True
)

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = iComformer(config).to(device)

# Load trained weights
checkpoint = torch.load(model_path, map_location=device)
model.load_state_dict(checkpoint['model'])
model.eval()

print(f"Model loaded from {model_path}")
print(f"Using device: {device}")

ImportError: attempted relative import with no known parent package

## 3. Generate Predictions for Test Data

Create a data loader for the test data and generate predictions.

In [ ]:
# Define the property we're predicting (same as in training script)
target_prop = "WF_bottom"

# Create test loader
test_loader = get_torch_test_loader(
    dataset="D2R2_surface_data",
    target=target_prop,
    batch_size=64,
    atom_features="cgcnn",
    cutoff=4.0,
    max_neighbors=25,
    id_tag="id",
    pyg_input=True,
    use_lattice=True,
    data_path=data_path,
    ids=train_test_val['id_test']
)

# Generate predictions
predictions = []
targets = []
ids = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        g, lg, _, target = batch
        output = model([g.to(device), lg.to(device), _.to(device)])
        
        # Get batch ID
        batch_ids = test_loader.dataset.ids
        ids.extend(batch_ids)
        
        # Collect predictions and targets
        output = output.cpu().numpy().tolist()
        if not isinstance(output, list):
            output = [output]
            
        target = target.cpu().numpy().flatten().tolist()
        if len(target) == 1:
            target = [target[0]]
            
        predictions.extend(output)
        targets.extend(target)

# Create predictions dataframe
results_df = pd.DataFrame({
    'id': ids,
    f'{target_prop}': targets,
    f'{target_prop}_pred': predictions
})

print(f"Generated {len(results_df)} predictions")
results_df.head()

## 4. Calculate Standardization Parameters

Calculate and verify the standardization parameters used during training.

In [ ]:
# Calculate mean and std for the target property using the training data
mean_train = train_data_df[target_prop].mean()
std_train = train_data_df[target_prop].std()

print(f"Training Data Statistics for {target_prop}:")
print(f"Mean = {mean_train:.6f}, Std = {std_train:.6f}")

## 5. Verify Standardization Parameters using Linear Regression

Following the approach in result_analysis.ipynb, we'll use linear regression to verify the standardization parameters.

In [ ]:
# Create a merged dataframe with both standardized and original values
merged_df = pd.merge(results_df, test_data_df[['id', target_prop]], on='id', suffixes=('_std', '_orig'))

# Calculate what the standardized values should be using our mean and std
merged_df[f'{target_prop}_calculated_std'] = (merged_df[f'{target_prop}_orig'] - mean_train) / std_train

# Calculate the difference between our calculated standardized values and those in the prediction file
diff = merged_df[f'{target_prop}_calculated_std'] - merged_df[f'{target_prop}_std']

print(f"Verification for {target_prop}:")
print(f"  Mean difference: {diff.mean():.6f}")
print(f"  Max absolute difference: {diff.abs().max():.6f}")
print(f"  Standard deviation of difference: {diff.std():.6f}")

# Determine if we need to recalculate standardization parameters
max_acceptable_diff = 0.05  # Threshold for acceptable difference
recalculate = diff.abs().max() > max_acceptable_diff

if recalculate:
    print("\nLarge discrepancy detected. Recalculating using linear regression...")
    
    # Use linear regression to find the actual parameters
    X = merged_df[f'{target_prop}_std'].values.reshape(-1, 1)
    y = merged_df[f'{target_prop}_orig'].values
    
    reg = LinearRegression().fit(X, y)
    a = reg.coef_[0]  # This is std_train
    b = reg.intercept_  # This is mean_train
    
    print(f"{target_prop}: y = {a:.6f} * x + {b:.6f}, R² = {reg.score(X, y):.6f}")
    
    print(f"\nOriginal mean_train: {mean_train:.6f}")
    print(f"Original std_train: {std_train:.6f}")
    print(f"Regression mean_train: {b:.6f}")
    print(f"Regression std_train: {a:.6f}")
    
    # Update mean and std if regression values are significantly different
    mean_diff = abs(mean_train - b)
    std_diff = abs(std_train - a)
    
    if mean_diff > 0.1 or std_diff > 0.1:
        print("\nUsing regression values for standardization parameters")
        mean_train = b
        std_train = a
    else:
        print("\nDifferences are small. Keeping original calculated values.")
else:
    print("\nStandardization parameters appear correct. No need to recalculate.")

## 6. De-standardize the Predictions

Now we'll convert the standardized predictions back to the original scale.

In [ ]:
# De-standardize the target and prediction values
results_df[f"{target_prop}_destd"] = results_df[target_prop] * std_train + mean_train
results_df[f"{target_prop}_pred_destd"] = results_df[f"{target_prop}_pred"] * std_train + mean_train

# Display the first few rows with de-standardized values
results_df.head()

## 7. Calculate Performance Metrics

Calculate MAE and MAPE for the predictions.

In [ ]:
# Calculate MAE between de-standardized predictions and original values
mae = mean_absolute_error(
    results_df[f"{target_prop}_destd"], 
    results_df[f"{target_prop}_pred_destd"]
)

# Calculate MAPE (avoiding division by zero)
non_zero_mask = results_df[f"{target_prop}_destd"] != 0
if non_zero_mask.sum() > 0:
    mape = np.mean(
        np.abs(
            (results_df[f"{target_prop}_destd"][non_zero_mask] - 
             results_df[f"{target_prop}_pred_destd"][non_zero_mask]) / 
            np.abs(results_df[f"{target_prop}_destd"][non_zero_mask])
        )
    ) * 100
else:
    mape = np.nan

print(f"Performance Metrics for {target_prop}:")
print(f"MAE: {mae:.6f}")
print(f"MAPE: {mape:.2f}%")

## 8. Visualize Results

Create visualizations to help understand the model's performance.

In [ ]:
# Create scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(
    results_df[f"{target_prop}_destd"], 
    results_df[f"{target_prop}_pred_destd"], 
    alpha=0.5
)

# Add perfect prediction line
min_val = min(results_df[f"{target_prop}_destd"].min(), results_df[f"{target_prop}_pred_destd"].min())
max_val = max(results_df[f"{target_prop}_destd"].max(), results_df[f"{target_prop}_pred_destd"].max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--')

plt.xlabel('Ground Truth')
plt.ylabel('Prediction')
plt.title(f'{target_prop} Predictions\nMAE: {mae:.4f}, MAPE: {mape:.2f}%')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save figure
output_dir = os.path.join(current_dir, 'output', 'D2R2_WF_bottom')
plt.savefig(os.path.join(output_dir, f'{target_prop}_prediction_vs_ground_truth.png'), dpi=300)
plt.show()

In [ ]:
# Create error histogram
plt.figure(figsize=(10, 8))

# Calculate errors
errors = results_df[f"{target_prop}_destd"] - results_df[f"{target_prop}_pred_destd"]

# Plot histogram
plt.hist(errors, bins=30, alpha=0.7)
plt.xlabel('Error (Ground Truth - Prediction)')
plt.ylabel('Frequency')
plt.title(f'{target_prop} Error Distribution\nMean: {errors.mean():.4f}, Std: {errors.std():.4f}')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Save figure
plt.savefig(os.path.join(output_dir, f'{target_prop}_error_distribution.png'), dpi=300)
plt.show()

## 9. Save Results

Save the predictions and metrics to files.

In [ ]:
# Save the results to a CSV file
output_file = os.path.join(output_dir, f'{target_prop}_predictions.csv')
results_df.to_csv(output_file, index=False)
print(f"Results saved to {output_file}")

# Save metrics to a JSON file
metrics = {
    'property': target_prop,
    'mae': float(mae),
    'mape': float(mape),
    'mean_error': float(errors.mean()),
    'std_error': float(errors.std()),
    'standardization': {
        'mean': float(mean_train),
        'std': float(std_train)
    }
}

metrics_file = os.path.join(output_dir, f'{target_prop}_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"Metrics saved to {metrics_file}")

## 10. Fix Missing Predictions Issue

Create a properly formatted predictions.json file for future use.

In [ ]:
# Create a proper predictions.json file similar to the multi-property format
predictions_list = []

for _, row in results_df.iterrows():
    predictions_list.append({
        'id': row['id'],
        'target': row[target_prop],
        'predictions': row[f'{target_prop}_pred']
    })

predictions_file = os.path.join(output_dir, 'predictions.json')
with open(predictions_file, 'w') as f:
    json.dump(predictions_list, f, indent=2)
print(f"Predictions JSON saved to {predictions_file}")